In [1]:
import pandas as pd

In [2]:
df_base = pd.read_csv("../data/00_baseline/raw_data.csv")
df_base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4246 entries, 0 to 4245
Data columns (total 4 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   startup_ID                   4246 non-null   int64 
 1   description_startupdetector  592 non-null    object
 2   startup_description          4112 non-null   object
 3   industry                     4246 non-null   object
dtypes: int64(1), object(3)
memory usage: 132.8+ KB


In [3]:
new_label = pd.read_csv('../data/03_mislabels/further_data/label_correction_checked_new.csv')
new_label.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5469 entries, 0 to 5468
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   startup_ID            5469 non-null   int64 
 1   is_label_issue        5469 non-null   bool  
 2   text                  5469 non-null   object
 3   given_label           5469 non-null   object
 4   suggested_label       5469 non-null   object
 5   decided_label         1391 non-null   object
 6   Data Error            29 non-null     object
 7   To check with sophia  8 non-null      object
 8   industry_new          5460 non-null   object
dtypes: bool(1), int64(1), object(7)
memory usage: 347.3+ KB


In [7]:
# print number of changed values for label issues
print(new_label.apply(lambda x: True if x['given_label'] != x['industry_new'] and x['is_label_issue'] else False, axis=1).value_counts())

False    4545
True      924
Name: count, dtype: int64


In [4]:
# save a raw dataframe for label correction only (excluding additional SD data from the data augmentation step)
cor_raw = df_base.merge(new_label[['startup_ID', 'industry_new']], 'left', 'startup_ID')
cor_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4498 entries, 0 to 4497
Data columns (total 5 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   startup_ID                   4498 non-null   int64 
 1   description_startupdetector  646 non-null    object
 2   startup_description          4352 non-null   object
 3   industry                     4498 non-null   object
 4   industry_new                 4498 non-null   object
dtypes: int64(1), object(4)
memory usage: 175.8+ KB


In [5]:
# print number of changed values in base dataset
print(cor_raw.apply(lambda x: True if x['industry'] != x['industry_new'] else False, axis=1).value_counts())
# drop old industry column and rename industry_new
cor_raw = cor_raw.drop(columns=['industry']).rename({"industry_new": "industry"}, axis=1)
cor_raw.info()

False    3718
True      780
Name: count, dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4498 entries, 0 to 4497
Data columns (total 4 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   startup_ID                   4498 non-null   int64 
 1   description_startupdetector  646 non-null    object
 2   startup_description          4352 non-null   object
 3   industry                     4498 non-null   object
dtypes: int64(1), object(3)
memory usage: 140.7+ KB


In [6]:
# save new raw dataframe
cor_raw.to_csv('../data/03_mislabels/raw_data.csv', index=False)